# Solution: Convolution and Pooling

[![Open In Colab <](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ShawnHymel/computer-vision-with-embedded-machine-learning/blob/master/2.1.4%20-%20Project%20-%20Convolution%20and%20Pooling/solution_convolution_and_pooling.ipynb)

Create a convolution filter to filter an image. Then, create a pooling function.

Run each of the cells paying attention to their contents and output. Fill out the necessary parts of the functions where you find the following comment:

```
# >>> ENTER YOUR CODE HERE <<<
```

Note that you may not use libraries other than those listed in the first cell to perform convolution and pooling.

You can see what the output images are supposed to look like [here](https://github.com/ShawnHymel/computer-vision-with-embedded-machine-learning/raw/master/2.1.4%20-%20Project%20-%20Convolution%20and%20Pooling/output-images.png).

If you get stuck or would like to compare answers, you can find [my solution here](https://colab.research.google.com/github/ShawnHymel/computer-vision-with-embedded-machine-learning/blob/master/2.1.4%20-%20Project%20-%20Convolution%20and%20Pooling/solution_convolution_and_pooling.ipynb).

Author: EdgeImpulse, Inc.<br>
Date: August 2, 2021<br>
License: [Apache-2.0](apache.org/licenses/LICENSE-2.0)<br>

In [1]:
### Import libraries

import os
import PIL
import requests
import math

import numpy as np
import matplotlib.pyplot as plt

In [2]:
### Download example image

# Image location and path
url = "https://github.com/ShawnHymel/computer-vision-with-embedded-machine-learning/raw/master/2.1.4%20-%20Project%20-%20Convolution%20and%20Pooling/resistor.png"
img_path = os.path.join("/content", "resistor.png")

# Download image
resp = requests.get(url)

# Write image to file
with open(img_path, 'wb') as f:
  f.write(resp.content)

ConnectTimeout: HTTPSConnectionPool(host='raw.githubusercontent.com', port=443): Max retries exceeded with url: /ShawnHymel/computer-vision-with-embedded-machine-learning/master/2.1.4%20-%20Project%20-%20Convolution%20and%20Pooling/resistor.png (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000029544633520>, 'Connection to raw.githubusercontent.com timed out. (connect timeout=None)'))

In [ ]:
### Open and view image

# Use PIL to open the image and convert it to grayscale
img = PIL.Image.open(img_path)
img = img.convert('L')

# Convert image to Numpy array
img = np.asarray(img)

# Show dimensions and view array as image
print(img.shape)
plt.imshow(img, cmap='gray', vmin=0, vmax=255)

## Part 1: Convolution

Your assignment is to write a function that convolves the image array (stored in the `img` variable) with a given kernel and stride. Assume valid padding (e.g. no padding). Note that the kernel will be given as a 2D Numpy array.

You are welcome to use for loops in this exercise, as it helps to see what's going on in the convolution operation. Bonus points if you do it without for loops (caveat: bonus points don't mean anything in this course).

You may find some of the following Numpy operations to be helpful:

* [numpy.arange()](https://numpy.org/doc/stable/reference/generated/numpy.arange.html)


In [ ]:
### Convolution function

def convolve(img, kernel, stride):

  # Compute dimensions of output image
  out_height = math.floor((img.shape[0] - kernel.shape[0]) / stride) + 1
  out_width = math.floor((img.shape[1] - kernel.shape[1]) / stride) + 1
  
  # Create blank output image
  convolved_img = np.zeros((out_height, out_width))

  # >>> ENTER YOUR CODE HERE <<<
  
  # Loop through each pixel in the output array. Note that this is not the most efficient way of
  # doing convolution, but it provides some insights into what's going on.
  for i in np.arange(0, out_height):
    for j in np.arange(0, out_width):

      # Set a temporary variable to 0
      accumulator = 0

      # Do element-wise multiplication and sum the result over the window/kernel
      for m in np.arange(0, kernel.shape[0]):
        for n in np.arange(0, kernel.shape[1]):
          accumulator += img[(stride * i) + m, (stride * j) + n] * kernel[m, n]

      # Set output image pixel to accumulator value
      convolved_img[i, j] = accumulator

  # Round all elements, convert to integers, and clamp to values between 0 and 255
  convolved_img = np.rint(convolved_img).astype(int)
  convolved_img = np.clip(convolved_img, 0, 255)

  return convolved_img

In [ ]:
### Test 1: Gaussian blur filter

# Define kernel
kernel = np.array([[1/16, 2/16, 1/16],
                   [2/16, 4/16, 2/16],
                   [1/16, 2/16, 1/16]])

# Call your convolve function (with a stride of 1)
out_img = convolve(img, kernel, 1)

# Show dimensions and view array as image
print(out_img.shape)
plt.imshow(out_img, cmap='gray', vmin=0, vmax=255)

In [ ]:
### Test 2: Edge detection

# Define kernel
kernel = np.array([[-1, -1, -1],
                   [-1, 8, -1],
                   [-1, -1, -1]])

# Call your convolve function (with a stride of 1)
out_img = convolve(img, kernel, 1)

# Show dimensions and view array as image
print(out_img.shape)
plt.imshow(out_img, cmap='gray', vmin=0, vmax=255)

In [ ]:
### Test 3: Sharpen with stride > 1

# Define kernel
kernel = np.array([[0, -1, 0],
                   [-1, 5, -1],
                   [0, -1, 0]])

# Call your convolve function (with a stride of 2)
out_img = convolve(img, kernel, 2)

# Show dimensions and view array as image
print(out_img.shape)
plt.imshow(out_img, cmap='gray', vmin=0, vmax=255)

## Part 2: Pooling

Your assignment is to write a function that performs max pooling on the input image array (stored in the `img` variable). Assume valid padding (e.g. no padding). Also, assume that the pool width determines the horizontal stride and the pool height determines the vertical stride.

You are welcome to use for loops in this exercise, as it helps to see what's going on in the convolution operation. Bonus points if you do it without for loops (caveat: bonus points don't mean anything in this course).

You may find some of the following Numpy operations to be helpful:

* [numpy.arange()](https://numpy.org/doc/stable/reference/generated/numpy.arange.html)

In [ ]:
### Max pooling function

def maxpooling(img, pool_height, pool_width):

  # Set stride amounts
  stride_y = pool_height
  stride_x = pool_width

  # Compute dimensions of output image
  out_height = math.floor((img.shape[0] - pool_height) / stride_y) + 1
  out_width = math.floor((img.shape[1] - pool_width) / stride_x) + 1
  
  # Create blank output image
  pooled_img = np.zeros((out_height, out_width))

  # >>> ENTER YOUR CODE HERE <<<

  # Loop through each pixel in the output array. Note that this is not the most efficient way of
  # doing convolution, but it provides some insights into what's going on.
  for i in np.arange(0, out_height):
    for j in np.arange(0, out_width):

      # Set output to the value of the first element in the window
      out_val = img[(stride_y * i), (stride_x * j)]

      # Look through each element in the window to find the max value
      for m in np.arange(0, pool_height):
        for n in np.arange(0, pool_width):
          out_val = max(out_val, img[(stride_y * i) + m, (stride_x * j) + n])

      # Set element in output array to max value
      pooled_img[i, j] = out_val

  # Round all elements, convert to integers, and clamp to values between 0 and 255
  pooled_img = np.rint(pooled_img).astype(int)
  pooled_img = np.clip(pooled_img, 0, 255)

  return pooled_img

In [ ]:
### Test 1: Max pool original image with pool size of (2, 3)

# Call your pooling function (with pool_size=(2, 3))
out_img = maxpooling(img, 2, 3)

# Show dimensions and view array as image
print(out_img.shape)
plt.imshow(out_img, cmap='gray', vmin=0, vmax=255)

In [ ]:
### Test 2: Detect edges and pool

# Define kernel
kernel = np.array([[-1, -1, -1],
                   [-1, 8, -1],
                   [-1, -1, -1]])

# Call your convolve function (with a stride of 1)
convolved_img = convolve(img, kernel, 1)

# Call your pooling function (with pool_size=(2, 2))
out_img = maxpooling(convolved_img, 2, 2)

# Show dimensions and view array as image
print(out_img.shape)
plt.imshow(out_img, cmap='gray', vmin=0, vmax=255)